In [ ]:
######################################################################################################################################################
#
#    Der Code extrahiert von den ersten 21000 Briefen die Metadaten und schreibt sie in eine CSV namens rom_korr_full.csv
#    Danach wird die Zeilen mit Werte "unbekannt" herausgefiltert und eine neue CSV namens rom_korr_website.csv geschrieben
#
######################################################################################################################################################

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

BASE_URL = "https://briefe-der-romantik.de/letters/view/"

def extract_metadata(letter_id):
    """
    Lädt eine Seite mit letter_id und parst daraus die Metadaten inklusive
    Name / Ort OHNE anhängendes '· GND' oder '· GeoNames'.
    """

    url = f"{BASE_URL}{letter_id}?left=text&query_id=6773ee556fe85"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch page for ID {letter_id}: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # Sucht den Metadaten-Bereich
    metadata_section = soup.find("div", id="metadata-tab-pane")
    if not metadata_section:
        print(f"No metadata found for ID {letter_id}")
        return None

    # Dictionary mit den englischen Labels (so wie sie in der Seite stehen)
    metadata = {
        "Date": None,
        "Sender": None,
        "Recipient": None,
        "Place of Dispatch": None,
        "Place of Destination": None,
        "Dispatch_GeoNames": None,
        "Destination_GeoNames": None,
        "link": url
    }

    # Schleife über alle <li>-Elemente
    for li in metadata_section.find_all("li"):
        label_span = li.find("span", class_="label")
        kdata_span = li.find("span", class_="kdata")
        if not label_span or not kdata_span:
            continue

        label_text = label_span.get_text(strip=True)
        kdata_raw_text = kdata_span.get_text(strip=True)

        # Nur das erste Segment vor dem ersten "·" übernehmen
        # \s*·\s* => trennt bei " · ", "·", " ·" usw.
        main_text = re.split(r'\s*·\s*', kdata_raw_text, maxsplit=1)[0].strip()

        # Falls das Label im Dictionary existiert
        if label_text in metadata:
            metadata[label_text] = main_text

            # GeoNames-Link ermitteln
            for a in kdata_span.find_all("a", href=True):
                if "geonames.org" in a["href"]:
                    if label_text == "Place of Dispatch":
                        metadata["Dispatch_GeoNames"] = a["href"]
                    elif label_text == "Place of Destination":
                        metadata["Destination_GeoNames"] = a["href"]

    return metadata


def scrape_letters(start_id, end_id):
    """
    Läuft über alle IDs von start_id bis end_id und sammelt
    die Metadaten in einer Liste aus Dicts.
    """
    letters_data = []
    for letter_id in range(start_id, end_id + 1):
        print(f"Processing letter ID {letter_id}...")
        metadata = extract_metadata(letter_id)
        if metadata:
            letters_data.append(metadata)
        time.sleep(0.5)  # Pause, damit der Server nicht überlastet wird
    return letters_data


def main():
    # Beispiel: Die ersten 21000 Briefe
    letters_data = scrape_letters(1, 21000)

    # In ein DataFrame laden und ausgeben
    df = pd.DataFrame(letters_data)
    print(df)
    return df


if __name__ == "__main__":
    df = main()
    # Bei Bedarf: df.head()


Processing letter ID 1...
Processing letter ID 2...
Processing letter ID 3...
Processing letter ID 4...
Processing letter ID 5...
Processing letter ID 6...
Processing letter ID 7...
Processing letter ID 8...
Processing letter ID 9...
Processing letter ID 10...
Processing letter ID 11...
Processing letter ID 12...
Processing letter ID 13...
Processing letter ID 14...
Processing letter ID 15...
Processing letter ID 16...
Processing letter ID 17...
Processing letter ID 18...
Processing letter ID 19...
Processing letter ID 20...
Processing letter ID 21...
Processing letter ID 22...
Processing letter ID 23...
Processing letter ID 24...
Processing letter ID 25...
Processing letter ID 26...
Processing letter ID 27...
Processing letter ID 28...
Processing letter ID 29...
Processing letter ID 30...
Processing letter ID 31...
Processing letter ID 32...
Processing letter ID 33...
Processing letter ID 34...
Processing letter ID 35...
Processing letter ID 36...
Processing letter ID 37...
Processing

In [2]:
# Save the data to a CSV file
df.to_csv("rom_korr_full.csv", index=False)

In [25]:
# Angenommen, dein gescrapter DataFrame heißt df und hat Spalten:
# ['Date', 'Sender', 'Recipient', 'Place of Dispatch', 'Place of Destination', ...]

# 1. Maske erstellen: True, wenn alle relevanten Felder = "Unknown"
mask_all_unknown = (
    (df["Date"] == "Unknown") &
    (df["Sender"] == "Unknown") &
    (df["Recipient"] == "Unknown") &
    (df["Place of Dispatch"] == "Unknown") &
    (df["Place of Destination"] == "Unknown")
)

# 2. Zusätzlich eine Maske für "Unknown" in den Spalten "Place of Dispatch" oder "Place of Destination"
mask_place_unknown = (
    (df["Place of Dispatch"] == "Unknown") |
    (df["Place of Destination"] == "Unknown")
)

# 3. Kombinierte Maske: Zeilen mit "Unknown" in allen relevanten Feldern oder in den neuen Spalten
mask_combined = mask_all_unknown | mask_place_unknown

# 4. Anzahl der betroffenen Zeilen ausgeben
count_combined_unknown = mask_combined.sum()  # sum() zählt 'True' als 1
print(f"Es gibt {count_combined_unknown} Zeilen, in denen entweder alle relevanten Felder oder 'Place of Dispatch' oder 'Place of Destination' 'Unknown' sind.")

# 5. Neuen DataFrame erzeugen, der nur Zeilen mit Metadaten (also nicht alles Unknown) enthält
rom_korr_website = df[~mask_combined].copy()


Es gibt 782 Zeilen, in denen diese 5 Felder 'Unknown' sind.
                           Date                       Sender  \
0   Donnerstag,  6. Januar 1791  August Wilhelm von Schlegel   
1  Mittwoch, 20. September 1797  August Wilhelm von Schlegel   
2         Samstag, 26. Mai 1798  August Wilhelm von Schlegel   
3    Mittwoch, 31. Oktober 1798  August Wilhelm von Schlegel   
4           [Mitte August 1801]             Sophie Bernhardi   

                     Recipient Place of Dispatch Place of Destination  \
0      Christian Gottlob Heyne         Göttingen            Göttingen   
1      Christian Gottlob Heyne              Jena            Göttingen   
2        Georg Joachim Göschen            Berlin              Leipzig   
3        Georg Joachim Göschen              Jena              Leipzig   
4  August Wilhelm von Schlegel            Berlin                 Jena   

                  Dispatch_GeoNames              Destination_GeoNames  \
0  https://www.geonames.org/2918632  https:

In [ ]:
# Save the data to a CSV file
rom_korr_website.to_csv("rom_korr_full_website.csv", index=False)

In [ ]:
############ Teil ab hier nur für eventuelle spätere Verarbeitung mit CSV ############

In [2]:
import pandas as pd

# CSV-Datei laden
file_name = "rom_korr_full.csv"  # Ersetze "deine_datei.csv" durch den tatsächlichen Dateinamen
df = pd.read_csv(file_name)

# Die ersten 5 Zeilen anzeigen, um sicherzustellen, dass die Daten korrekt geladen wurden
print(df.head())

                           Date                       Sender  \
0   Donnerstag,  6. Januar 1791  August Wilhelm von Schlegel   
1  Mittwoch, 20. September 1797  August Wilhelm von Schlegel   
2         Samstag, 26. Mai 1798  August Wilhelm von Schlegel   
3    Mittwoch, 31. Oktober 1798  August Wilhelm von Schlegel   
4           [Mitte August 1801]             Sophie Bernhardi   

                     Recipient Place of Dispatch Place of Destination  \
0      Christian Gottlob Heyne         Göttingen            Göttingen   
1      Christian Gottlob Heyne              Jena            Göttingen   
2        Georg Joachim Göschen            Berlin              Leipzig   
3        Georg Joachim Göschen              Jena              Leipzig   
4  August Wilhelm von Schlegel            Berlin                 Jena   

                  Dispatch_GeoNames              Destination_GeoNames  \
0  https://www.geonames.org/2918632  https://www.geonames.org/2918632   
1  https://www.geonames.org/28

In [3]:
# Angenommen, dein gescrapter DataFrame heißt df und hat Spalten:
# ['Date', 'Sender', 'Recipient', 'Place of Dispatch', 'Place of Destination', ...]

# 1. Maske erstellen: True, wenn alle relevanten Felder = "Unknown"
mask_all_unknown = (
    (df["Date"] == "Unknown") &
    (df["Sender"] == "Unknown") &
    (df["Recipient"] == "Unknown") &
    (df["Place of Dispatch"] == "Unknown") &
    (df["Place of Destination"] == "Unknown")
)

# 2. Zusätzlich eine Maske für "Unknown" in den Spalten "Place of Dispatch" oder "Place of Destination"
mask_place_unknown = (
    (df["Place of Dispatch"] == "Unknown") |
    (df["Place of Destination"] == "Unknown")
)

# 3. Kombinierte Maske: Zeilen mit "Unknown" in allen relevanten Feldern oder in den neuen Spalten
mask_combined = mask_all_unknown | mask_place_unknown

# 4. Anzahl der betroffenen Zeilen ausgeben
count_combined_unknown = mask_combined.sum()  # sum() zählt 'True' als 1
print(f"Es gibt {count_combined_unknown} Zeilen, in denen entweder alle relevanten Felder oder 'Place of Dispatch' oder 'Place of Destination' 'Unknown' sind.")

# 5. Neuen DataFrame erzeugen, der nur Zeilen mit Metadaten (also nicht alles Unknown) enthält
rom_korr_website = df[~mask_combined].copy()


Es gibt 16607 Zeilen, in denen entweder alle relevanten Felder oder 'Place of Dispatch' oder 'Place of Destination' 'Unknown' sind.


In [4]:
print(rom_korr_website.head())

                           Date                       Sender  \
0   Donnerstag,  6. Januar 1791  August Wilhelm von Schlegel   
1  Mittwoch, 20. September 1797  August Wilhelm von Schlegel   
2         Samstag, 26. Mai 1798  August Wilhelm von Schlegel   
3    Mittwoch, 31. Oktober 1798  August Wilhelm von Schlegel   
4           [Mitte August 1801]             Sophie Bernhardi   

                     Recipient Place of Dispatch Place of Destination  \
0      Christian Gottlob Heyne         Göttingen            Göttingen   
1      Christian Gottlob Heyne              Jena            Göttingen   
2        Georg Joachim Göschen            Berlin              Leipzig   
3        Georg Joachim Göschen              Jena              Leipzig   
4  August Wilhelm von Schlegel            Berlin                 Jena   

                  Dispatch_GeoNames              Destination_GeoNames  \
0  https://www.geonames.org/2918632  https://www.geonames.org/2918632   
1  https://www.geonames.org/28

In [5]:
# Save the data to a CSV file
rom_korr_website.to_csv("rom_korr_full_website.csv", index=False)